**PREPARATION**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.auto import tqdm
import sys

import ddpm
import datasets

import os
import trimesh

c:\Users\jgual\anaconda3\envs\your_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

print(f"PyTorch version Installed: {torch.__version__}\nTorchvision version Installed: {torchvision.__version__}\n")
if not torch.__version__.startswith("1.11"):
    print("you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it"
          " might cause dependency and compatibility issues.")
if not torchvision.__version__.startswith("0.12"):
    print("you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it"
          " might cause dependency and compatibility issues.")

PyTorch version Installed: 1.13.1
Torchvision version Installed: 0.14.1+cpu

you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it might cause dependency and compatibility issues.
you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it might cause dependency and compatibility issues.


**DATASET & DATALOADER**

In [9]:
# DATA NORMALIZATION FUNCTION PRIOR TO DATASET GENERATION

# Import dataset class
from grasp_object_dataset import graspDataset
from torch.utils.data import DataLoader

# Function to obtain mean, std, max, and min of given dataset
def get_mean_std(main_dir, object_dir, dataset):

    # Allocate variables
    full_joints = []

    # Load all samples - only joint values!
    for sample in tqdm(dataset):
        joints = sample[0]
        full_joints.append(joints)

    mean = np.mean(full_joints, axis = 0)
    std = np.std(full_joints, axis = 0)
    max = np.max(full_joints, axis = 0)
    min = np.min(full_joints, axis = 0)
    

    return mean, std, max, min


In [54]:
# DATASET AND DATALOADER DEFINITION

# Import dataset class
from grasp_object_dataset import graspDataset
from torch.utils.data import DataLoader

# Directories of dataset grasps + objects: set to a small version of it to overfit!! Full version found in dataset_XX_full
main_dir = './dataset_grasps/'
object_dir = './dataset_objects/'

# Generate dataset with all dataset samples
main_dataset = graspDataset(main_dir, object_dir, mode = 'train', split = {'train': 1, 'val': 0, 'test': 0}, normalization=None, transform_joint = None, transform_object = None)

# Calculate mean, std, max, min of full dataset
mean_std_max_min = list(get_mean_std(main_dir, object_dir, main_dataset))
# Print not norm data
print('NOT NORMALIZED: ', mean_std_max_min)

# Datasets definition
train_dataset = graspDataset(main_dir, object_dir, mode = 'train', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)
val_dataset = graspDataset(main_dir, object_dir, mode = 'val', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)
test_dataset = graspDataset(main_dir, object_dir, mode = 'test', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, normalization=mean_std_max_min)#, transform_joint = None, transform_object = None)

# Comparison for normalized dataset
mean_std_max_min_2 = list(get_mean_std(main_dir, object_dir, train_dataset))
print('NORMALIZED: ', mean_std_max_min_2)

# Dataloader definition
train_dataloader = DataLoader(train_dataset , batch_size=10, shuffle=False, num_workers=2, drop_last=False)
val_dataloader = DataLoader(val_dataset , batch_size=10, shuffle=False, num_workers=2, drop_last=False)
test_dataloader = DataLoader(test_dataset , batch_size=10, shuffle=False, num_workers=2, drop_last=False)

100%|██████████| 2/2 [00:00<00:00, 251.07it/s]


NOT NORMALIZED:  [array([ 0.05989712,  0.35345435,  0.77724156,  0.44774483,  0.00311129,
        0.36377603,  0.88474086,  0.35140082, -0.03054787,  0.27029156,
        0.82724717,  0.49818423,  0.07623768, -0.09648404,  0.09449551,
        0.87018171,  0.39562432,  0.12631763,  0.98540276,  0.02745842,
       -0.20692084, -0.01524943, -0.41709315, -0.02226195,  0.04890059,
        0.08755917, -0.02611019, -0.0812342 ]), array([4.82669584e-02, 4.11015749e-03, 5.36862910e-02, 4.19571251e-02,
       8.22551106e-03, 2.27979124e-02, 3.46976817e-02, 3.50584090e-02,
       4.13688356e-02, 3.11455503e-02, 1.70304477e-02, 1.02818578e-01,
       4.19151410e-03, 1.43269151e-02, 3.51050757e-02, 7.06234574e-03,
       1.49358362e-02, 3.86578515e-02, 1.82699561e-02, 6.56746645e-02,
       8.43219459e-04, 1.47095334e-02, 8.03794683e-01, 2.78834817e-02,
       1.57361461e+00, 8.63660127e-04, 1.96129391e-02, 1.53720379e-04]), array([ 1.08164079e-01,  3.57564509e-01,  8.30927849e-01,  4.89701957e-01,


100%|██████████| 1/1 [00:00<?, ?it/s]

NORMALIZED:  [array([-1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1., -1.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([-1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1., -1.]), array([-1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1., -1.])]


In [5]:
# DATASET INDICES

# train_dataset[sample][index]
# index = 0 --> joint angles
# index = 1 --> label (grasp type) - one hot vector
# index = 2 --> matrix distances
# index = 3 --> file name

#print(train_dataset[0][1])

**DIFFUSION MODEL**

In [107]:
# HYPERPARAMETERS FOR TUNING

# model
hidden_size = 128
hidden_layers = 3
emb_size= 128
time_emb= "sinusoidal"
input_emb= "sinusoidal"

# Noise scheduler
num_timesteps = 100
beta_schedule= 'linear'

# optimizer
learning_rate = 1e-3

#training
num_epochs = 20

In [108]:
model = ddpm.MLP(
        hidden_size= hidden_size,
        hidden_layers=hidden_layers,
        emb_size= emb_size,
        time_emb= time_emb,
        input_emb= input_emb
        )

noise_scheduler = ddpm.NoiseScheduler(
        num_timesteps=num_timesteps,
        beta_schedule=beta_schedule)

optimizer = torch.optim.AdamW(
        model.parameters(),
        lr= learning_rate,
    )

global_step = 0
frames = []
losses = []
print("Training model...")
for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(total=len(train_dataloader))
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in enumerate(train_dataloader):
        # Variables
        joint_angles_batch = batch[0] 
        label_one_hot = batch[1]
        mat_distances = batch[2] 

        # Noise and timesteps
        noise = torch.randn(joint_angles_batch.shape)
        timesteps = torch.randint(0, noise_scheduler.num_timesteps, (joint_angles_batch.shape[0],)).long()

        noisy = noise_scheduler.add_noise(joint_angles_batch, noise, timesteps)
        noisy = noisy.double()
        noise_pred = model(noisy, timesteps, label_one_hot, mat_distances)
        loss = F.mse_loss(noise_pred, noise)
        loss.backward(loss)

        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        logs = {"loss": loss.detach().item(), "step": global_step}
        losses.append(loss.detach().item())
        progress_bar.set_postfix(**logs)
        global_step += 1

    progress_bar.close()


"""
    if epoch % config.save_images_step == 0 or epoch == config.num_epochs - 1:
        # generate data with the model to later visualize the learning process
        model.eval()
        sample = torch.randn(config.eval_batch_size, 2)
        timesteps = list(range(len(noise_scheduler)))[::-1]
        for i, t in enumerate(tqdm(timesteps)):
            t = torch.from_numpy(np.repeat(t, config.eval_batch_size)).long()
            with torch.no_grad():
                residual = model(sample, t)
            sample = noise_scheduler.step(residual, t[0], sample)
        frames.append(sample.numpy())

print("Saving model...")
outdir = f"exps/{config.experiment_name}"
os.makedirs(outdir, exist_ok=True)
torch.save(model.state_dict(), f"{outdir}/model.pth")

print("Saving images...")
imgdir = f"{outdir}/images"
os.makedirs(imgdir, exist_ok=True)
frames = np.stack(frames)
xmin, xmax = -6, 6
ymin, ymax = -6, 6
for i, frame in enumerate(frames):
    plt.figure(figsize=(10, 10))
    plt.scatter(frame[:, 0], frame[:, 1])
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.savefig(f"{imgdir}/{i:04}.png")
    plt.close()
    
print("Saving loss as numpy array...")
np.save(f"{outdir}/loss.npy", np.array(losses))
print("Saving frames...")
np.save(f"{outdir}/frames.npy", frames)
"""

Training model...


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\jgual\Desktop\BEMP\Advanced Deep Learning For Robotics\ADLR_project\ADLRProject\ddpm.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_tensor= torch.tensor(label, dtype=torch.float32)
Epoch 19: 100%|██████████| 1/1 [00:05<00:00,  5.09s/it, loss=0.663, step=19]


'\n    if epoch % config.save_images_step == 0 or epoch == config.num_epochs - 1:\n        # generate data with the model to later visualize the learning process\n        model.eval()\n        sample = torch.randn(config.eval_batch_size, 2)\n        timesteps = list(range(len(noise_scheduler)))[::-1]\n        for i, t in enumerate(tqdm(timesteps)):\n            t = torch.from_numpy(np.repeat(t, config.eval_batch_size)).long()\n            with torch.no_grad():\n                residual = model(sample, t)\n            sample = noise_scheduler.step(residual, t[0], sample)\n        frames.append(sample.numpy())\n\nprint("Saving model...")\noutdir = f"exps/{config.experiment_name}"\nos.makedirs(outdir, exist_ok=True)\ntorch.save(model.state_dict(), f"{outdir}/model.pth")\n\nprint("Saving images...")\nimgdir = f"{outdir}/images"\nos.makedirs(imgdir, exist_ok=True)\nframes = np.stack(frames)\nxmin, xmax = -6, 6\nymin, ymax = -6, 6\nfor i, frame in enumerate(frames):\n    plt.figure(figsize=(

**MODEL INFERENCE --> EVALUATION OF RESULTS**

In [109]:
# Directory for grasping validation
main_dir = './dataset_grasps/'
object_dir = './dataset_objects/'

# Grasp code for which we have overfitting
grasp_code = 'core-bottle-1ae823260851f7d9ea600d1a6d9f6e07'

# Load matrix distance for given object
matriz_distancias = np.load(os.path.join(object_dir+grasp_code+".npy"), allow_pickle=True)*0.11
matriz_distancias = torch.from_numpy(matriz_distancias)
matriz_distancias = matriz_distancias.reshape(1, 50, 50, 50)

# Define matrix for given object
label = torch.tensor([1, 0, 0])
label = label.reshape(1, 3)

In [110]:
# Model inference
model.eval()

eval_batch_size = 1
num_timesteps = 50
plot_step = 5

noise_scheduler = ddpm.NoiseScheduler(num_timesteps=num_timesteps)
sample = torch.randn(eval_batch_size, 28)
timesteps = list(range(num_timesteps))[::-1]

samples = []
steps = []

for i, t in enumerate(tqdm(timesteps)):
    t = torch.from_numpy(np.repeat(t, eval_batch_size)).long()
    with torch.no_grad():
        residual = model(sample, t, label, matriz_distancias)
        
    sample = noise_scheduler.step(residual, t[0], sample)
    if (i + 1) % plot_step == 0:
        samples.append(sample.numpy())
        steps.append(i + 1)

# Generated test grasp is saved in samples[-1]

100%|██████████| 50/50 [00:05<00:00,  9.27it/s]


In [105]:
main_dataset[0][0]
train_dataset[0][0]

array([-1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1., -1.])

In [106]:
# Print of joint angles for predicted grasp
print(samples[-1][0])


# Set sample to predicted test sample
sample = samples[-1][0]


# Return normalized values to the initial values
sample = (sample + 1)/2
sample = sample * (mean_std_max_min[2]- mean_std_max_min[3]) + mean_std_max_min[3]

print(sample)

[ 0.56097275 -1.348307   -0.2814042  -1.2367761  -0.6833564   0.35016468
 -0.34673825  1.585526    0.4555008   0.759703    0.5786873  -0.5539173
 -0.79644275 -1.2521323  -1.1657857   1.0199156   1.4242939  -2.295847
 -1.5134664   0.23400554  0.53017765 -0.11692504 -0.13669054 -4.2730665
 -1.9446757  -0.24939169 -1.1451687  -0.2737894 ]
[ 8.69735666e-02  3.47912597e-01  7.62134010e-01  3.95853261e-01
 -2.50966701e-03  3.71759051e-01  8.72709847e-01  4.06986841e-01
 -1.17043318e-02  2.93952925e-01  8.37102477e-01  4.41231246e-01
  7.28993812e-02 -1.14423228e-01  5.35705146e-02  8.77384706e-01
  4.16897346e-01  3.75651177e-02  9.57751799e-01  4.28266573e-02
 -2.06473784e-01 -1.69693412e-02 -5.26964290e-01 -1.41409919e-01
 -3.01126947e+00  8.73437851e-02 -4.85703095e-02 -8.12762888e-02]


**VISUALIZATION OF RESULT**

In [46]:
import random
from utils.hand_model_lite import HandModelMJCFLite
import numpy as np
import transforms3d
import torch
import trimesh

In [68]:
mesh_path = './test_files/meshdata/'
data_path = './test_files/dexgraspnet/'


use_visual_mesh = False

hand_file = "mjcf/shadow_hand_vis.xml" if use_visual_mesh else "mjcf/shadow_hand_wrist_free.xml"

joint_names = [
    'robot0:FFJ3', 'robot0:FFJ2', 'robot0:FFJ1', 'robot0:FFJ0',
    'robot0:MFJ3', 'robot0:MFJ2', 'robot0:MFJ1', 'robot0:MFJ0',
    'robot0:RFJ3', 'robot0:RFJ2', 'robot0:RFJ1', 'robot0:RFJ0',
    'robot0:LFJ4', 'robot0:LFJ3', 'robot0:LFJ2', 'robot0:LFJ1', 'robot0:LFJ0',
    'robot0:THJ4', 'robot0:THJ3', 'robot0:THJ2', 'robot0:THJ1', 'robot0:THJ0'
]
translation_names = ['WRJTx', 'WRJTy', 'WRJTz']
rot_names = ['WRJRx', 'WRJRy', 'WRJRz']

In [64]:
# Load hand file
hand_model = HandModelMJCFLite(
    hand_file,
    "mjcf/meshes")

In [92]:
# Load grasp original code and object
grasp_code = 'core-bottle-1ae823260851f7d9ea600d1a6d9f6e07'
grasp_data = np.load(
    os.path.join(data_path, grasp_code+".npy"), allow_pickle=True)
object_mesh_origin = trimesh.load(os.path.join(
    mesh_path, grasp_code, "coacd/decomposed.obj"))

{'robot0:FFJ3': -0.24548496305942535, 'robot0:FFJ2': 0.6087559461593628, 'robot0:FFJ1': 0.7188290357589722, 'robot0:FFJ0': 0.2019304633140564, 'robot0:MFJ3': -0.17577102780342102, 'robot0:MFJ2': 0.38283923268318176, 'robot0:MFJ1': 1.1127177476882935, 'robot0:MFJ0': 0.403972327709198, 'robot0:RFJ3': -0.07210764288902283, 'robot0:RFJ2': 0.1400628387928009, 'robot0:RFJ1': 0.9195994138717651, 'robot0:RFJ0': 0.41342633962631226, 'robot0:LFJ4': 0.18373392522335052, 'robot0:LFJ3': 0.025994563475251198, 'robot0:LFJ2': 0.13054056465625763, 'robot0:LFJ1': 0.9286351203918457, 'robot0:LFJ0': 0.47552821040153503, 'robot0:THJ4': -0.0618005096912384, 'robot0:THJ3': 1.1335370540618896, 'robot0:THJ2': 0.041698239743709564, 'robot0:THJ1': -0.13501165807247162, 'robot0:THJ0': -2.997061756104813e-06, 'WRJRx': 3.0913850314695135, 'WRJRy': 0.2305903394170629, 'WRJRz': -2.3657796059005416, 'WRJTx': -0.043085746467113495, 'WRJTy': 0.09048712253570557, 'WRJTz': 0.1175876185297966}


In [111]:
# Select first grasph - could be any
index = 0

# Set sample to predicted test sample
sample = samples[-1][0]

# Return normalized values to the initial values
sample = (sample + 1)/2
sample = sample * (mean_std_max_min[2]- mean_std_max_min[3]) + mean_std_max_min[3]


# Substitute values in grasp for the predicted ones
qpos = grasp_data[index]['qpos']
i=0
for key in qpos:
    qpos[key] = sample[i]
    i += 1


# Plot the results
rot = np.array(transforms3d.euler.euler2mat(
    *[qpos[name] for name in rot_names]))
rot = rot[:, :2].T.ravel().tolist()
hand_pose = torch.tensor([qpos[name] for name in translation_names] + rot + [qpos[name]
                         for name in joint_names], dtype=torch.float, device="cpu").unsqueeze(0)
hand_model.set_parameters(hand_pose)
hand_mesh = hand_model.get_trimesh_data(0)
object_mesh = object_mesh_origin.copy().apply_scale(0.11)#grasp_data[index]["scale"])

(hand_mesh+object_mesh).show()